In [1]:
#important library
import pandas as pd
import glob

In [2]:
# loading all csv file names
files = glob.glob("csvs_per_year/*.csv")

In [3]:
# reading csv files
for index,file in enumerate(files):
    files[index] = pd.read_csv(file)

In [4]:
# merging all csv files
df = pd.concat(files)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3808224 entries, 0 to 69095
Data columns (total 19 columns):
 #   Column   Dtype  
---  ------   -----  
 0   date     object 
 1   BEN      float64
 2   CO       float64
 3   EBE      float64
 4   MXY      float64
 5   NMHC     float64
 6   NO_2     float64
 7   NOx      float64
 8   OXY      float64
 9   O_3      float64
 10  PM10     float64
 11  PXY      float64
 12  SO_2     float64
 13  TCH      float64
 14  TOL      float64
 15  station  int64  
 16  PM25     float64
 17  NO       float64
 18  CH4      float64
dtypes: float64(17), int64(1), object(1)
memory usage: 581.1+ MB
